## Run this first for imports and MakeMCQ function.

In [57]:
import docx
import random
import math
import import_ipynb
import QuizFunctions as QF
from docx import Document
from docx.enum.text import WD_ALIGN_PARAGRAPH

topics = ['Collecting Like Terms','Decimal Addition','Multiplying Terms','Adding and Multiplying Terms', 'Adding Negatives',
         'Multiplying Negatives', 'Adding and Multiplying Negatives']

def info():
    print('To make a quiz, type in: MakeQuiz(Topic, Number of Questions, Interleaved (True or False), Difficulty Array)')
    print('Topics are...')
    for topic in topics:
        print(topic)
    print('Number of Questions is just an integer for you to type in.')
    print('Interleaved - typing True will prompt for more info on what you want to interleave with. False means it will remain within the topic.')
    print('Difficulty Array - questions are of difficulty 1 (simplest) to 5 (heavily interleaved). Typically enter [1,2,3,4,5]')


#This is a sequence of programs around floats. Fed in to prettify function.
def AutoRoundFloat(Flt):
    Flt += 0.0000000001
    StartIndex = str(Flt).find('.')
    EndIndex = str(Flt)[StartIndex+1:].find('00000')
    OutStr = str(Flt)[:EndIndex+StartIndex+1]
    if OutStr[-1] == '.':
        OutStr = OutStr[:-1]
    return OutStr

def HasFloat(Str):
    bOut = False
    while '999999' in Str:
        x = Str.find('999999')
        while x != 0:
            if Str[x].isnumeric():
                x -= 1
            elif Str[x] == '.':
                bOut = True
                break
            else:
                break
        Str = Str[:Str.find('999999')] + Str[Str.find('999999')+6:]
    while '000000' in Str:
        x = Str.find('000000')
        while x != 0:
            if Str[x].isnumeric():
                x -= 1
            elif Str[x] == '.':
                bOut = True
                break
            else:
                break
        Str = Str[:Str.find('000000')] + Str[Str.find('000000')+6:]
    return bOut

def GetFloatIndexes(String):
    FoundFloat = False
    FrontIndex = 0
    String = String + ' '
    while FoundFloat == False:
        if String[0].isnumeric():
            x = 0
            while String[x].isnumeric():
                x += 1
            if String[x] == '.':
                x += 1
                if String[x:x+10].isnumeric() and ('999999' in String[x:x+10] or '000000' in String[x:x+10]):
                    while String[x].isnumeric():
                        x += 1
                    BackIndex = x + FrontIndex
                    FoundFloat = True
                else:
                    while String[x].isnumeric():
                        x += 1
                    String = String[x+1:]
                    FrontIndex += x+1
            else:
                String = String[x+1:]
                FrontIndex += x + 1
        else: 
            String = String[1:]
            FrontIndex += 1
    return [FrontIndex,BackIndex]
            
def RoundFloatsInString(String):
    while HasFloat(String):
        Indexes = GetFloatIndexes(String)
        Flt = float(String[Indexes[0]:Indexes[1]])
        String = String[:Indexes[0]] + str(AutoRoundFloat(Flt))+ String[Indexes[1]:]
    return String

#This handles stuff so that I can code questions in using syntax like ^ and get the exponent.
#Also changes i.e. 1c to c, -1c to -c, 0c to 0.
#When refactoring, place this in QuizFunctions and have this doc draw from QuizFunctions
def prettify(string):
    if type(string) != str:
        string = str(string)
    #Superscripts
    normal = "ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz0123456789+-=()"
    super_s = "ᴬᴮᶜᴰᴱᶠᴳᴴᴵᴶᴷᴸᴹᴺᴼᴾQᴿˢᵀᵁⱽᵂˣʸᶻᵃᵇᶜᵈᵉᶠᵍʰᶦʲᵏˡᵐⁿᵒᵖ۹ʳˢᵗᵘᵛʷˣʸᶻ⁰¹²³⁴⁵⁶⁷⁸⁹⁺⁻⁼⁽⁾"
    while '^' in string:
        index = string.find('^')
        if string[index+1]!='(':
            string = string[:index+1] + '(' + string[index+1] + ')' + string[index+2:]
        res = string.maketrans(''.join(normal),''.join(super_s))
        bracket1 = index + string[index:].find('(')
        bracket2 = index + string[index:].find(')')
        if string[bracket1+1:bracket2] == '1':
            letterstoreplace = ''
        else:
            letterstoreplace = string[bracket1+1:bracket2].translate(res)
        string = string[:index]+letterstoreplace+string[bracket2+1:]
    varlist = ['a','b','c','d','e','f','x','y','z']
    #Management of terms eg. 1a - 1b
    for var in varlist:
        if string[:2] == '1'+var:
            string = string[1:]
        elif string[:3] == '-1'+var:
            string = '-'+string[2:]
        elif string[:2] == '0'+var:
            string = string[3:]
        while ' 1' + var in string:
            loc = string.find(' 1'+var)
            string = string[:loc+1]+var+string[loc+3:]
        while '-1' + var in string:
            loc = string.find('-1'+var)
            string = string[:loc+1]+var+string[loc+3:]
        while ' 0' + var in string:
            loc = string.find(' 0' + var)
            string = string[:loc] + string[loc+3:]
    #General cleanups
    if string[:2]=='+ ':
        string = string[2:]
    
    #Adding negatives where the intention would have been to simply subtract:
    while '+ -' in string:
        index = string.find('+ -')
        string = string[:index] + '– ' + string[index + 3:]
    while ' - ' in string:
        index = string.find(' - ')
        string = string[:index] + ' – ' + string[index + 3:]
    stringlocal = string + '  '
    while '.0 ' in stringlocal:
        index = stringlocal.find('.0 ')
        stringlocal = stringlocal[:index] + stringlocal[index + 2:-1]
        print(string)
        print(stringlocal)
        string = stringlocal
    
    #Float rounding
    string = RoundFloatsInString(string)
    return string
    
#Given a question and list of answers, will output an MCQ. Qno is the question number.
def MakeMCQ(Question, AnsList, Qno, document):
    r = 2
    c = len(AnsList)
    document.add_paragraph()
    document.add_paragraph(str(Qno) + '. ' +prettify(Question))
    table = document.add_table(rows=r, cols=c)
    table.style = 'Table Grid'
    for x in range(r):
        for y in range(c):
            if(x == 0):
                cell = table.rows[x].cells[y]
                cell_paragraph = cell.paragraphs[0]
                cell_paragraph.text = Opts[y]
                cell_paragraph.alignment = WD_ALIGN_PARAGRAPH.CENTER
            else:
                cell = table.rows[x].cells[y]
                cell_paragraph = cell.paragraphs[0]
                cell_paragraph.text = prettify(AnsList[y])
                cell_paragraph.alignment = WD_ALIGN_PARAGRAPH.CENTER

def MakeQuiz(Topic,Qs,ILBool,bShadowQuiz,DifficultyArr,QuizTitle):
    document = Document()
    if bShadowQuiz == True:
        iterations = 2
    else:
        iterations = 1
    for it in range(iterations):
        Answers = []
        TopicEntered = False
        document.add_heading(Topic + ' Quiz', 1)
        if Topic == 'Collecting Like Terms':
            Quiz = QGen_CollectLikeTerms(Qs,DifficultyArr)
            TopicEntered = True

        elif Topic == 'Decimal Addition':
            Quiz = QGen_DecimalAddition(Qs,DifficultyArr)
            TopicEntered = True

        elif Topic == 'Multiplying Terms':
            Quiz = QGen_MultiplyTerms(Qs,DifficultyArr)
            TopicEntered = True

        elif Topic == 'Adding and Multiplying Terms':
            Quiz = QGen_AddMultTerms(Qs,DifficultyArr)
            TopicEntered = True
            
        elif Topic == 'Adding Negatives':
            Quiz = QF.HandleNegativesQuiz(QGen_AddNegatives(Qs,DifficultyArr))
            QF.HandleNegativesQuiz(Quiz)
            TopicEntered = True
        
        elif Topic == 'Multiplying Negatives':
            Quiz = QF.HandleNegativesQuiz(QGen_TimesNegatives(Qs,DifficultyArr))
            QF.HandleNegativesQuiz(Quiz)
            TopicEntered = True
            
        elif Topic == 'Adding and Multiplying Negatives':
            Quiz = QGen_AddTimesNegatives(Qs,DifficultyArr)
            QF.HandleNegativesQuiz(Quiz)
            TopicEntered = True
            
        if TopicEntered:
            #
            #
            #IMPORTANT: You should change the below function to 'for x in range(Qs)'
            #It's only like this now because only one style of collecting like terms exists.
            #
            #
            for x in range(int(Qs)):
                MakeMCQ(Quiz[0][x][0],Quiz[1][x],x+1,document)
                Answers.append(Quiz[2][x][1])
        document.add_paragraph()
        document.add_paragraph(['Answers:'])

        i = 1
        for answer in Answers:
            document.add_paragraph([str(i)+'. ']+[answer])
            i+= 1
    footer = document.sections[0].footer
    footerpara = footer.paragraphs[0]
    footerpara.text = "Copyright 2022 (your name here)"
    document.save(QuizTitle + '.docx')
    print("Document "+ QuizTitle + ".docx has been created.")
    

def SetQPS(NumOfQs,DiffList):
    qps = []
    for i in range(len(DiffList)):
        qps.append(math.floor(NumOfQs/len(DiffList)))
    if NumOfQs%len(DiffList)!=0:
        for i in range(NumOfQs%len(DiffList)):
            qps[i]= qps[i]+1
    return qps

#Is this function needed still?
def MCShuffle(MC,Removes):
    a = [MC[0]]
    MC.pop(0)
    random.shuffle(MC)
    for i in range (Removes):
        MC.pop(0)
    MC = a + MC
    random.shuffle(MC)
    if NoneStatement in MC:
        MC.pop(MC.index(NoneStatement))
        MC.append(NoneStatement)
    return MC

#This shuffles and specifically places a nonestatement at the back, if it is to be found.
def Shuffle(MC):
    random.shuffle(MC)
    if NoneStatement in MC:
        index = MC.index(NoneStatement)
        MC.pop(index)
        MC.append(NoneStatement)
    return MC

#Takes MCQ list (answer index 0) and has a small chance for answer to be nonestatement, or for options to be nonestatement.
def NoneStatementChance(MCAnsList):
    if random.randint(1,15)==15:
        MCAnsList = [NoneStatement] + MCAnsList[1:]
    elif random.randint(1,12)==12:
        MCAnsList.pop(random.randint(1,4))
        MCAnsList.append(NoneStatement)
    return MCAnsList
    

def IndexToABC(Index):
    if Index==0: return 'a'
    elif Index==1: return 'b'
    elif Index==2: return 'c'
    elif Index==3: return 'd'
    else: return 'e'

    #rounds a list of numbers.
def roundlist(fltlist,dp):
    OutList = []
    for x in fltlist:
        OutList.append(round(x,dp))
    return OutList


#blend accepts an array of QGen data and blends them together. 
#With topics A & B, it will sort questions like a1,a2,b1,b2,a3,a4 etc.
#Note that blend only works for arrays that are similarly sized.
def blend(ArrOfLists):
    Topics = len(ArrOfLists)
    Questions = 0
    OutQ = [[],[],[]]
    for x in range(Topics):
        Questions += len(ArrOfLists[x][0])
    
    while Questions != 0:
        
        for n in range(Topics):
            if len(ArrOfLists[n][0]) >=2:
                for i in range(2):
                    OutQ[0].append(ArrOfLists[n][0].pop(0))
                    OutQ[1].append(ArrOfLists[n][1].pop(0))
                    OutQ[2].append(ArrOfLists[n][2].pop(0))
                    Questions -= 1
            elif len(ArrOfLists[n][0]) == 1:
                OutQ[0].append(ArrOfLists[n][0].pop(0))
                OutQ[1].append(ArrOfLists[n][1].pop(0))
                OutQ[2].append(ArrOfLists[n][2].pop(0))
                Questions -= 1
    return OutQ
    
                
NoneStatement = 'none of these'                
Opts = ['A','B','C','D','E']
Ques = 'What country has capital city Paris?'
Ans = ['France','Germany','Italy','Netherlands']                
#document.add_heading('This is a generated quiz', 1)
#MakeMCQ(Ques,Ans)
    
#document.save('Demo.docx')

In [58]:
# Generators are below.
# REFACTORING: This codebase is really messy/inefficient/hard to replicate.
# If you continue to use this, declare an object instead and 
# break it apart into little functions to improve readability.
# Scaling became a serious issue here because of the copy/pasting that I did at the beginning of this project.


# Collecting like terms. Refactor - this is way too large?
def QGen_CollectLikeTerms(NumOfQs, DiffList):
    
    qps = SetQPS(NumOfQs,DiffList)
    OutQ = [[],[],[]]
    SentStarter = ['Simplify ','Collect these like terms: ']
    Variables = ['a','b','c','d','e','f','x','y']
    MCQOpts = 5
    Ans = ''
    
    #do each of these QPS[0] times. Style1 Question Gen below.
    if DiffList[0] == 1:
        for z in range(qps[0]):
            QLong = random.choice(SentStarter)
            Q = ''
            Var = random.choice(Variables)
            NumOfTerms = random.randint(2,4)
            if random.choice([False,True]):
                for i in range(NumOfTerms):
                    Q = Q + Var + ' + '
                Q = Q[:-3]
                Ans = str(NumOfTerms) + Var
                OutQ[0].append([QLong + Q,Q])
                OutQ[2].append([Ans])
                #Answer always goes first.
                MC = [Ans,
                      Var + '^' + str(NumOfTerms),
                      (str(NumOfTerms-1)+Var),
                      (str(NumOfTerms+random.randint(1,2))+Var),
                      (str(NumOfTerms-random.randint(1,2))+Var),
                      str(NumOfTerms) + ' + ' + Var,
                     NoneStatement]
                #trying to remove options from MC down to MCQOpts integer.
                MC = MCShuffle(MC,len(MC)-MCQOpts)
                OutQ[1].append(MC)
                OutQ[2][-1].append(IndexToABC(OutQ[1][-1].index(Ans)))
            else:
                #coefficients
                Coefficients = []
                Coefficients.append(random.randint(1,6))
                for i in range(NumOfTerms-1):
                    Coefficients.append(random.choice([(random.randint(1,5)),(random.randint(1,12)),(random.randint(-5,-1))]))

                if Coefficients[0] == -1:
                    Q = Q + '-' + Var
                elif Coefficients[0] == 1:
                    Q = Q + Var
                else: 
                    Q = Q + str(Coefficients[0]) + Var
                for coef in Coefficients[1:]:    
                    if coef<0:
                        if coef == -1: 
                            Q = Q + ' – ' + Var
                        else:
                            Q = Q + ' – ' + str(coef)[1:] + Var
                    elif coef == 1:
                        Q = Q + ' + ' + Var
                    else:
                        Q = Q + ' + ' + str(coef) + Var
                Ans = str(sum(Coefficients))+Var
                OutQ[0].append([QLong + Q,Q])
                OutQ[2].append([Ans])
                MC = [Ans,
                      str(sum(Coefficients))+Var + '^' + str(NumOfTerms),
                      str(sum(Coefficients)) + ' + ' + Var,
                      str(-sum(Coefficients))+Var,
                      str(sum(Coefficients)),
                      str(sum(Coefficients)+random.randint(1,5)),
                      str(sum(Coefficients)-random.randint(1,5)),
                      str(sum(Coefficients)+random.randint(1,5))+Var,
                      str(sum(Coefficients)-random.randint(1,5))+Var,
                      str(sum(Coefficients)+random.randint(1,15))+Var,
                      str(sum(Coefficients)-random.randint(1,30))+Var,
                      (str(NumOfTerms-1)+Var),
                      str(NumOfTerms) + ' + ' + Var,
                      NoneStatement]
                MC = MCShuffle(MC,len(MC)-MCQOpts)
                OutQ[1].append(MC)
                OutQ[2][-1].append(IndexToABC(OutQ[1][-1].index(Ans)))
        qps.pop(0)
        DiffList.pop(0)
        
    
    #Style2: 2a + 2b + 5a + 3b, subtractions
    if DiffList[0] == 2:
        for z in range(qps[0]):
            QLong = random.choice(SentStarter)
            Q = ''
            #Code changed from here
            Var = random.choice(Variables[:-1])
            Indx = Variables.index(Var)
            Var2 = random.choice(Variables[Indx:])
            #To here
            NumOfTerms = 4
            Coefficients = []
            #setting coefficients a, b, a, b so that first A&B are larger than second.
            acoef = random.randint(2,15)
            bcoef = random.randint(2,15)
            Coefficients += [acoef,bcoef,random.randint(1,acoef),random.randint(1,bcoef)]
            
            #Creation of the question.
            Q = Q + str(Coefficients[0]) + Var
            Q = Q + ' + ' + str(Coefficients[1]) + Var2
            if random.choice([True,False]) == True:
                Q = Q + ' + ' + str(Coefficients[2]) + Var
            else:
                Q = Q + ' – ' + str(Coefficients[2]) + Var
                Coefficients[2] = -Coefficients[2]
            if random.choice([True,False]) == True:
                Q = Q + ' + ' + str(Coefficients[3]) + Var2
            else:
                Q = Q + ' – ' + str(Coefficients[3]) + Var2
                Coefficients[3] = -Coefficients[3]
            
            #Case where nonestatement is the answer
            if random.choice([True,False,False,False,False,False])==True:
                Ans = NoneStatement
            else:
                #Answer
                Ans = str(Coefficients[0]+Coefficients[2])+Var+' + '+str(Coefficients[1]+Coefficients[3])+Var2
            OutQ[0].append([QLong + Q,Q])
            OutQ[2].append([Ans])
            
            if Coefficients[2]<0:
                #Case where third term is negative, fourth positive i.e. 10a + 10b - 3a + 2b
                MC = [Ans,
                      str(sum(Coefficients))+Var+Var2,
                      str(sum(Coefficients)),
                      str(Coefficients[0]+Coefficients[2])+Var+' + '+str(Coefficients[1]-Coefficients[3])+Var2,
                      str(Coefficients[0]+Coefficients[2])+Var+'^2'+' + '+str(Coefficients[1]+Coefficients[3])+Var2+'^2']
            else:
                MC = [Ans,
                      str(sum(Coefficients))+Var+Var2,
                      str(sum(Coefficients)),
                      str(Coefficients[0]+Coefficients[2]+random.randint(-1,4))+Var+' + '+str(Coefficients[1]+Coefficients[3])+Var2,
                      str(Coefficients[0]+Coefficients[2])+Var+'^2'+' + '+str(Coefficients[1]+Coefficients[3])+Var2+'^2']
            
            #Adding small chance for a nonestatement to appear
            if Ans!=NoneStatement and random.choice([True,False,False,False])==True:
                MC.append(NoneStatement)
            MC = MCShuffle(MC,len(MC)-MCQOpts)
            OutQ[1].append(MC)
            OutQ[2][-1].append(IndexToABC(OutQ[1][-1].index(Ans)))
        qps.pop(0)
        DiffList.pop(0)
    
    #Style3: Include squared terms x & x^2 or a, b, ab
    if DiffList[0] == 3:
        for z in range(qps[0]):
            QLong = random.choice(SentStarter)
            Q = ''
            Var = random.choice(Variables[:-1])
            Indx = Variables.index(Var)
            Var2 = random.choice(Variables[Indx:])
            while Var2 == Var:
                Var2 = random.choice(Variables)
            Var3 = random.choice([Var+Var2,Var+'^2'])
            NumOfTerms = random.randint(4,5)
            #setting coefficients a, b, a, b so that first A&B are larger than second.
            acoef = random.randint(2,20)
            bcoef = random.randint(2,20)
            abcoef = random.randint(2,20)
            #Different paths for if there are 4 coefficients (will use Var & Var3) or 5 (Varx2, Var2x1, Var3x2)
            if NumOfTerms==4:
                Coefficients = [acoef,abcoef,random.randint(1,acoef),random.randint(1,abcoef)]
                Q = Q + str(Coefficients[0]) + Var
                Q = Q + ' + ' + str(Coefficients[1]) + Var3
                if random.choice([True,False]) == True:
                    Q = Q + ' + ' + str(Coefficients[2]) + Var
                else:
                    Q = Q + ' – ' + str(Coefficients[2]) + Var
                    Coefficients[2] = -Coefficients[2]
                if random.choice([True,False]) == True:
                    Q = Q + ' + ' + str(Coefficients[3]) + Var3
                else:
                    Q = Q + ' – ' + str(Coefficients[3]) + Var3
                    Coefficients[3] = -Coefficients[3]
            
            else:
                Coefficients = [acoef,bcoef,abcoef,random.randint(1,acoef),random.randint(1,abcoef)]
                Q = Q + str(Coefficients[0]) + Var
                Q = Q + ' + ' + str(Coefficients[1]) + Var2
                Q = Q + ' + ' + str(Coefficients[2]) + Var3
                if random.choice([True,False]) == True:
                    Q = Q + ' + ' + str(Coefficients[3]) + Var
                else:
                    Q = Q + ' – ' + str(Coefficients[3]) + Var
                    Coefficients[3] = -Coefficients[3]
                if random.choice([True,False]) == True:
                    Q = Q + ' + ' + str(Coefficients[4]) + Var3
                else:
                    Q = Q + ' – ' + str(Coefficients[4]) + Var3
                    Coefficients[4] = -Coefficients[4]
            
            #Case where nonestatement is the answer
            if random.choice([True,False,False,False,False,False])==True:
                Ans = NoneStatement
            elif NumOfTerms==4:
                #Answer
                Ans = str(Coefficients[0]+Coefficients[2])+Var+' + '+str(Coefficients[1]+Coefficients[3])+Var3
            else:
                Ans = str(Coefficients[0]+Coefficients[3])+Var+' + '+str(Coefficients[1])+Var2+' + '+str(Coefficients[2]+Coefficients[4])+Var3
            OutQ[0].append([QLong + Q,Q])
            OutQ[2].append([Ans])
            
            if NumOfTerms==4:
                if Coefficients[2]<0:
                    #Case where third term is negative, fourth positive i.e. 10a + 10b - 3a + 2b
                    MC = [Ans,
                          str(sum(Coefficients))+Var+Var3,
                          str(sum(Coefficients))+Var+'^2'+Var3[-1],
                          str(sum(Coefficients)),
                          str(Coefficients[0]+Coefficients[2])+Var+' + '+str(Coefficients[1]-Coefficients[3])+Var3,
                          str(Coefficients[0]+Coefficients[2])+Var+'^2'+' + '+str(Coefficients[1]+Coefficients[3])+Var3+'^2']
                else:
                    MC = [Ans,
                          str(sum(Coefficients))+Var+Var3,
                          str(sum(Coefficients))+Var+'^2'+Var3[-1],
                          str(sum(Coefficients)),
                          str(Coefficients[0]+Coefficients[2]+random.randint(-1,4))+Var+' + '+str(Coefficients[1]+Coefficients[3])+Var3,
                          str(Coefficients[0]+Coefficients[2])+Var+'^2'+' + '+str(Coefficients[1]+Coefficients[3])+Var3+'^2']
            else:
                #this is for 5 terms a, b, ab, a, ab
                MC = [Ans,
                      str(sum(Coefficients))+Var+Var+Var+Var+Var2+Var2+Var2,
                      str(sum(Coefficients))+Var+'^4'+Var2+'^3',
                      str(Coefficients[2]-Coefficients[4])+Var3 + ' + ' + str(Coefficients[0]+Coefficients[3])+Var+' + '+str(Coefficients[1])+Var2,
                      str(Coefficients[2]+Coefficients[4]+random.randint(1,2))+Var3 + ' + ' + str(Coefficients[0]+Coefficients[3]+random.randint(0,1))+Var+' + '+str(Coefficients[1])+Var2,
                      str(Coefficients[2]+Coefficients[4]-random.randint(1,2))+Var3 + ' + ' + str(Coefficients[0]+Coefficients[3]+random.randint(0,1))+Var+' + '+str(Coefficients[1])+Var2]

            #Adding small chance for a nonestatement to appear
            if Ans!=NoneStatement and random.choice([True,False,False,False])==True:
                MC.append(NoneStatement)
            MC = MCShuffle(MC,len(MC)-MCQOpts)
            OutQ[1].append(MC)
            OutQ[2][-1].append(IndexToABC(OutQ[1][-1].index(Ans)))
        qps.pop(0)
        DiffList.pop(0)
        
    #Style4: Include squared terms x & x^2 or a, b, ab, and can feature negative answers. More paramaterized questions here.
    if DiffList[0] == 4:
        for z in range(qps[0]):
            QLong = random.choice(SentStarter)
            Q = ''
            Var = random.choice(Variables[:-1])
            Indx = Variables.index(Var)
            Var2 = random.choice(Variables[Indx:])
            while Var2 == Var:
                Var2 = random.choice(Variables)
            #for first question type
            a = str(random.randint(5,12))
            b = str(random.randint(-11,-1))
            c = str(random.randint(-25,-5))
            d = str(int(a) - int(c))
            #for second question type
            e = str(random.randint(1,6))
            f = str(random.randint(7,35))
            g = str(random.randint(7,35))
            h = str(random.randint(-6,-1))
            #coeffs of answer
            i = str(int(e) - int(g))
            j = str(int(f) + int(h))
            Var3 = random.choice([Var+Var2,Var+'^2',Var+'^2'+Var2,Var+Var2+'^2'])
            PreMadeQandA = random.choice([[a+Var3 + ' + ' + '('+b+Var+') – '+'('+c+Var3+')',
                                           #mcqlist
                                          [str(int(a)+int(c))+Var3+' + '+b+Var,
                                           str(int(a)+int(c))+Var3+' + '+b[1:]+Var,
                                           d+Var3+' + '+b[1:]+Var,
                                           '-'+d+Var3+' + '+b+Var],
                                          d+Var3+' '+b+Var],
                                          [e+Var + ' + ' + f + Var2 + ' – ' + g + Var + ' + (' + h + Var2 + ')',
                                           #mcqlist
                                           [i + Var + ' + ' + str(int(f) - int(h)) + Var2, 
                                            str(int(i)+int(j))+Var+Var2, 
                                            str(int(e) + int(g))+Var+ ' + ' + j + Var2,
                                            str(int(e) + int(g))+Var+' + '+ str(int(f) - int(h)) + Var2],
                                           i + Var + ' + ' + j + Var2]])
            Q = PreMadeQandA[0]
            if random.choice([True,False,False,False,False,False])==True:
                Ans = NoneStatement
            else:
                Ans = PreMadeQandA[2]
            OutQ[0].append([QLong + Q,Q])
            OutQ[2].append([Ans])
            
            MC = [Ans]+PreMadeQandA[1]
            
            #Adding small chance for a nonestatement to appear
            if Ans!=NoneStatement and random.choice([True,False,False,False])==True:
                MC.append(NoneStatement)
            MC = MCShuffle(MC,len(MC)-MCQOpts)
            OutQ[1].append(MC)
            OutQ[2][-1].append(IndexToABC(OutQ[1][-1].index(Ans)))
        qps.pop(0)
        DiffList.pop(0)
    #Style5: As before but with decimals.
    if len(DiffList) != 0:
        for z in range(qps[0]):
            QLong = random.choice(SentStarter)
            Q = ''
            Var = random.choice(Variables[:-1])
            Indx = Variables.index(Var)
            Var2 = random.choice(Variables[Indx:])
            while Var2 == Var:
                Var2 = random.choice(Variables)
            #for first question type
            a = str(random.randint(5,12)+random.choice([0.1,0.4,0.5,0.6,0.9,0.02,0.04,0.07]))
            b = str(random.randint(-11,-1)+random.choice([0.1,0.4,0.5,0.6,0.9,0.02,0.04,0.07]))
            c = str(random.randint(-25,-5)+random.choice([0.1,0.4,0.5,0.6,0.9,0.02,0.04,0.07]))
            d = str(float(a) - float(c))
            #for second question type
            e = str(random.randint(1,6)+random.choice([0.1,0.4,0.5,0.6,0.9,0.02,0.04,0.07]))
            f = str(random.randint(7,35)+random.choice([0.1,0.4,0.5,0.6,0.9,0.02,0.04,0.07]))
            g = str(random.randint(7,35)+random.choice([0.1,0.4,0.5,0.6,0.9,0.02,0.04,0.07]))
            h = str(random.randint(-6,-1)+random.choice([0.1,0.4,0.5,0.6,0.9,0.02,0.04,0.07]))
            #coeffs of answer
            i = str(round(float(e) - float(g),2))
            j = str(round(float(f) + float(h),2))
            Var3 = random.choice([Var+Var2,Var+'^2',Var+'^2'+Var2,Var+Var2+'^2'])
            PreMadeQandA = random.choice([[a+Var3 + ' + ' + '('+b+Var+') – '+'('+c+Var3+')',
                                           #mcqlist
                                          [str(round((float(a)+float(c)),2))+Var3+' + '+b+Var,
                                           str(round((float(a)+float(c)),2))+Var3+' + '+b[1:]+Var,
                                           d+Var3+' + '+b[1:]+Var,
                                           '-'+d+Var3+' + '+b+Var],
                                          d+Var3+' '+b+Var],
                                          [e+Var + ' + ' + f + Var2 + ' – ' + g + Var + ' + (' + h + Var2 + ')',
                                           #mcqlist
                                           [i + Var + ' + ' + str(float(f) - float(h)) + Var2, 
                                            str(round((float(i)+float(j)),2))+Var+Var2, 
                                            str(round((float(e) + float(g)),2))+Var+ ' + ' + j + Var2,
                                            str(round((float(e) + float(g)),2))+Var+' + '+ str(float(f) - float(h)) + Var2],
                                           i + Var + ' + ' + j + Var2]])
            Q = PreMadeQandA[0]
            if random.choice([True,False,False,False,False,False])==True:
                Ans = NoneStatement
            else:
                Ans = PreMadeQandA[2]
            OutQ[0].append([QLong + Q,Q])
            OutQ[2].append([Ans])
            
            MC = [Ans]+PreMadeQandA[1]
            
            #Adding small chance for a nonestatement to appear
            if Ans!=NoneStatement and random.choice([True,False,False,False])==True:
                MC.append(NoneStatement)
            MC = MCShuffle(MC,len(MC)-MCQOpts)
            OutQ[1].append(MC)
            OutQ[2][-1].append(IndexToABC(OutQ[1][-1].index(Ans)))
        qps.pop(0)
        DiffList.pop(0)
    return OutQ

Question generators below: Collecting like terms.

In [59]:
#Creating a quiz on decimal addition. Also thinking about functions that will be reused.
#Will plan it out carefully this time; want a much smaller function.


#Returns power difference. 10 means that a is a power of 10 greater than b (we only care about digits - eg. 90 vs 2 is fine).
def PowerDifference(a,b):
    a = str(a)
    b = str(b)
    aPower = 1
    bPower = 1
    if len(a) != 1:
        if '.' not in a:
            aPower = 10**(len(a)-1)
        elif a[:2] == '0.':
            while a[0] == '0':
                aPower = aPower/10
                a = str(float(a)*10)
        else:
            aPower = 10**(len(a[:a.find('.')])-1)
    if len(b) != 1:
        if '.' not in b:
            bPower = 10**(len(b)-1)
        elif b[:2] == '0.':
            while b[0] == '0':
                bPower = bPower/10
                b = str(float(b)*10)
        else:
            bPower = 10**(len(b[:b.find('.')])-1)
    return aPower/bPower
    
#Coding in the misconceptions
#example: call Miscons_DeecimalAddition([0.7,5,0.2],[+,-])

def Miscons_DecimalAddition(TermList,Operations):
    OutList = []
    
    a = TermList[0]
    b = TermList[1]
    
    #Partial calculation of answer
    if Operations[0] == '+':
        Ans = a + b
    else:
        Ans = a - b

    if len(TermList) == 2:
        OutList.append(Ans)
        ()
        #Misconceptions -> a + pdb, a/pd + b, where pd is the difference in powers of 10.
        pd = PowerDifference(a,b)
        lena = 0
        lenb = 0
        if '.' in str(a):
            lena = len(str(a)[str(a).find('.'):])
        if '.' in str(b):
            lenb = len(str(b)[str(b).find('.'):])
        if Operations[0] == '+':
            OutList.append(a + b*pd)
            OutList.append(round((a/pd + b),max(lena,lenb)))
        else:
            OutList.append(round((a - b*pd),max(lena,lenb)))
            OutList.append(round((a/pd - b),max(lena,lenb)))
        #Add decimals as if they are integers, and refactor i.e. 6.2 + 2.12 = 8.14
        if '.' in str(a) and '.' in str(b):
            aint = math.floor(a)
            bint = math.floor(b)
            adec = int(str(a-aint)[2:])
            bdec = int(str(b-bint)[2:])
            OutList.append(round((float(str(aint + bint) + '.' + str(adec + bdec))),max(len(str(adec)),len(str(bdec)))))
        #For additions, subtract 1 from the answer (no carrying). For subtractions, add 1.
        if Operations[0] == '+':
            OutList.append(Ans + 1)
        else: OutList.append(Ans - 1)
        
        #Randomised answer based on answer
        OutList.append(Ans + 0.1*random.randint(-20,20))
        if OutList[0] == OutList[-1]: 
            OutList[-1] = OutList[-1]+0.1

    else: #There are 3 terms.
        c = TermList[2]
        if Operations[1] == '+':
            Ans = Ans + c
        else:
            Ans = Ans - c
        OutList.append(Ans)
        
        #Misconceptions -> a + pdb, a/pd + b, where pd is the difference in powers of 10.
        d = max(a,b,c)
        pda = PowerDifference(d,a)
        pdb = PowerDifference(d,b)
        pdc = PowerDifference(d,c)
        OutList.append(a*pda + b*pdb + c*pdc)
        
        #Add decimals as if they are integers, and refactor i.e. 6.2 + 2.12 = 8.14
        adec = 0
        if '.' in str(a): 
            adec = int(str(a)[str(a).find('.')+1:])
        bdec = 0
        if '.' in str(b): 
            bdec = int(str(b)[str(b).find('.')+1:])
        cdec = 0
        if '.' in str(c): 
            cdec = int(str(c)[str(c).find('.')+1:])
        aint = int(a - adec)
        bint = int(b - bdec)
        cint = int(c - cdec)
        OutList.append(round(float(str(aint + bint + cint) + '.' + str(adec + bdec + cdec)),len(str(max(adec, bdec, cdec)))))
        
        if Operations[0] == '-' and Operations[1] == '+':
            OutList.append(a - (b + c))
        
        #Randomised answer based on answer
        OutList.append(Ans + 0.1*random.randint(-20,20))
        if OutList[0] == OutList[-1]: 
            OutList[-1] = OutList[-1]+0.1
        
        OutList.append(Ans + random.choice([-1,1]))
    #DO NOT randomise yet or add nonevalue, consider that step carefully.
    while len(OutList)>5:
        OutList.pop(random.randint(1,4))
    return OutList

def QGen_DecimalAddition(NumOfQs, DiffList):
    qps = SetQPS(NumOfQs,DiffList)
    OutQ = [[],[],[]]
    SentStarter = ['Work out ','Calculate ', 'Find the total of ']
    MCQOpts = 5
    Ans = ''
    DiffListCounter = min(DiffList)
    
    #Style 1: Add or subtract single decimal to larger whole num      of type  a + b
    #Style 2: Add or subtract two decimals (no negatives to appear anywhere)
    #Style 3: + - two decimals of form 0.ab + 0.0cd where b&c could be 0.
    
    #Note this will fail to work if you have a difflist of, say, [1,3,4,5].
    #If this happens, fix by doing as two while loops: (1) while difflist counter <4, (2) while DiffListCounter in DIffList
    #(so that if the difflistcounter isn't in the list, you can then increment without creating questions.)
    while DiffListCounter < 4 and DiffListCounter in DiffList:
        for z in range(qps[0]):
            if DiffListCounter == 1:
                a = random.randint(1,10) * random.choice([1,1,1,2,5,10])
                b = random.randint(1,9) * random.choice([0.1,0.1,0.01,0.12,0.13])
                op = random.choice(['+','-'])

            elif DiffListCounter == 2:
                a = random.randint(1,10)*random.choice([1,1,1,2,5,10])+random.randint(1,9)*random.choice([0.1,0.1,0.01,0.11,0.15])
                b = random.randint(1,9) * random.choice([0.1,0.1,0.01,0.12,0.13]) + random.randint(1,3)
                op = random.choice(['+','-'])

            elif DiffListCounter == 3:
                a = random.randint(1,9)*0.1 + random.randint(0,9)*0.01
                b = random.randint(0,9)*0.01 + random.randint(1,9)*0.001
                op = random.choice(['+','-'])

            #Swap a&b for some variation when +
            if op == '+' and random.choice([True,False]):
                c = a
                a = b
                b = c

            Q = f"{a} {op} {b}"
            MC = Miscons_DecimalAddition([a,b],[op])

            #Possible this could be its own function, down to the appends. See if you re-use it a lot.
            Ans = MC[0]
            MC = NoneStatementChance(MC)
            MCAns = MC[0] #Small chance this could now be the nonestatement
            print(MC)
            MC = Shuffle(MC)
            MCAns = IndexToABC(MC.index(MCAns)) #Taking the answer, converting to index, then converting to a letter.
            #appends
            OutQ[0].append([random.choice(SentStarter) + Q, Q])
            OutQ[1].append(MC)
            OutQ[2].append([Ans,MCAns])
        qps.pop(0)
        DiffListCounter += 1
    
    #Style 4: Three decimals with order of operations. a - b + c or a + c - b.
    #Style 5: Style 4, but b is a bracket term (). 
    DiffListCounter = 4
    while DiffListCounter < 6 and DiffListCounter in DiffList:
        for z in range(qps[0]):
            a = random.randint(5,15) + random.randint(1,9)*0.1
            b1 = random.randint(1,9)+random.randint(1,9)*random.choice([0,0.1])
            b2 = random.randint(1,9)*random.choice([0.1,0.01])
            bop = random.choice(['+','-'])
            if bop == '+':
                b = b1+b2
            else:
                b = b1-b2
            c = random.randint(5,15) + random.randint(1,9)*0.1
            op = random.choice([['+','-'],['-','+']])
            
            if DiffListCounter == 4: 
                if op[0]=='+':
                    Q = f"{a} {op[0]} {c} {op[1]} {b}"
                else:
                    Q = f"{a} {op[0]} {b} {op[1]} {c}"
            else:
                if op[0]=='+':
                    Q = f"{a} {op[0]} {c} {op[1]} ({b1} {bop} {b2})"
                else:
                    Q = f"{a} {op[0]} ({b1} {bop} {b2}) {op[1]} {c}"
            
            if op[0] =='+':
                MC = Miscons_DecimalAddition([a,c,b],op)
            else:
                MC = Miscons_DecimalAddition([a,b,c],op)
            
            Ans = MC[0]
            MC = NoneStatementChance(MC)
            MCAns = MC[0] #Small chance this could now be the nonestatement
            MC = Shuffle(MC)
            MCAns = IndexToABC(MC.index(MCAns)) #Taking the answer, converting to index, then converting to a letter.
            #appends
            OutQ[0].append([random.choice(SentStarter) + Q, Q])
            OutQ[1].append(MC)
            OutQ[2].append([Ans,MCAns])
        qps.pop(0)
        DiffListCounter += 1
    return OutQ

In [60]:
#QGen_MultiplyingTerms
#Style 3 style 2, but terms have a power of 1-8 with 2 being more likely.
#Style 4 style 2, but there are now three terms, one of which is an 5ab or 7cd or something like this. (focus on a, b, c only)
#Style 5 Now terms or powers may be negative, and answers as well.
#Qn variations (what are the powers of {})

#Common misconceptions: alternate MC answers as such: Coefficients added, powers added (incorrect)
#Coefficients added, powers multiplied (incorrect)
#Coefficients multiplied, powers added (correct)
#All multiplied (incorrect)

#Get correct number multiplication ()
#for powers, multiply them rather than add them

#Note that when returning, answer is always the first MC option.
def Style_MultiplyTerms(StyleInt):
    if StyleInt == 1:
        var1 = random.choice(['a','b','c','d','e','f'])
        c1 = random.choice([1,random.randint(1,12)])
        c2 = random.randint(2,12)
        Q = random.choice(["Simplify","Work out", "Multiply", "Find the product of"])
        if random.choice([True,True,False]):
            return [[f"{Q} {c1}{var1} × {c2}",f"{c1}{var1} × {c2}"],[str(c1*c2)+var1,str(c1+c2)+var1,c1+c2,c1*c2,random.choice([f"{c1*c2}{var1}^2",f"{c1+c2}{var1}^2"])]]
        elif random.choice([True,False,False]):
            return [[f"What word is used to describe {c1}{var1} × {c2}?",f"{c1}{var1} × {c2}"],['expression','term','variable','coefficient','equation']]
        elif random.choice([True,False]):
            return [[f"In the expression {c1}{var1} × {c2}, what is {c1}?",f"{c1}{var1} × {c2}, what is {c1}?"],['coefficient','term','variable','expression','equation']]
        else:
            return [[f"In the expression {c1}{var1} × {c2}, what is {var1}?",f"{c1}{var1} × {c2}, what is {var1}?"],['variable','term','coefficient','expression','equation']]
    
    #All other styles below
    
    if StyleInt <= 3:
        var1 = random.choice(['a','b','c','d','e','f'])
        var2 = random.choice([var1,var1,var1,var1,'a','b','c','d','e','f'])
    else:
        var1 = random.choice(['a','b','c'])
        var2 = random.choice(['d','e','f'])
        rneg = 1 #Only ever used in style 5, but factored into style 4's code.
        
    c1 = random.choice([random.randint(1,12),random.randint(1,15)*random.randint(2,5),1])
    c2 = random.randint(2,12)
    
    if StyleInt == 5:
        c1 = c1 * random.choice([1,-1])
        c2 = c2 * random.choice([1,-1])
        rneg = random.choice([1,-1])
    
    cprod = c1*c2
    csum = c1+c2
    QStart = random.choice(["Simplify","Work out", "Multiply", "Find the product of"])
    
    if StyleInt>2:
        pow1 = random.choice([1,2,random.randint(2,8)])
        pow2 = random.choice([2,2,2,random.randint(3,4)])
        if pow1==pow2:
            pow2 += 1
        
        if StyleInt == 5:
            pow1 = pow1 * random.choice([-1,1])
            pow2 = pow2 * random.choice([-1,1])
        
        powsum = pow1 + pow2
        powprod = pow1 * pow2
    else:
        pow1 = 1
        pow2 = 1

    if StyleInt >= 4:
        c3 = random.randint(1,3)
        var3 = var1 + var2
        cprod = cprod*c3
        csum = csum + c3
        pow3 = [random.randint(1,3),random.randint(1,3)]
        powsum = powsum + sum(pow3)
        powprod = powprod * sum(pow3)
        Q = f"{c1}{var1}^({pow1}) × {c2}{var2}^({pow2}) × {c3}{var1}^({pow3[0]}){var2}^({pow3[1]})"
    else:
        Q = f"{c1}{var1}^({pow1}) × {c2}{var2}^({pow2})"
    
    if StyleInt == 2:
        if var1 == var2:
            return[[QStart + ' ' + Q, Q],
                   [f"{cprod}{var1}^2",
                    f"{csum}{var1}^2",
                    f"{cprod}{var1}",
                    f"{csum}{var1}",
                    random.choice([f"{cprod+1}{var1}^2",f"{csum+random.randint(-1,8)}{var1}^2"])]]
        else:
            return [[QStart + ' ' + Q, Q],
                   [f"{cprod}{var1}{var2}",
                    f"{csum}{var1}{var2}",
                    f"{cprod}{var1}{random.choice(['',f'^{var2}'])}",
                    f"{csum}{var1}{random.choice(['',f'^{var2}'])}",
                    random.choice([f"{cprod+1}{var1}{var2}",f"{csum+random.randint(-1,8)}{var1}{var2}"])]]
    elif StyleInt == 3:
        #Common misconceptions for same variable: multiply power, add power. multiply coefficients, add coefficients.
        #if variables are different: append variables & bracket them, add power. append & bracket, mult. maintain power. etc.
        if var1 == var2:
            return[[QStart + ' ' + Q, Q],
                       [f"{cprod}{var1}^({powsum})",
                        f"{csum}{var1}^({powsum})",
                        f"{cprod}{var1}^({powprod})",
                        f"{csum}{var1}^({powprod})",
                        f"({random.choice([cprod,csum])}{var1})^({random.choice([powsum,powprod])})"]]
        else:
            return[[QStart + ' ' + Q, Q],
                       [f"{cprod}{var1}^({pow1}){var2}^({pow2})",
                        f"{csum}{var1}^({pow1}){var2}^({pow2})",
                        f"{cprod}{var1}{var2}^{random.choice([powprod,powsum])}",
                        f"{csum}{var1}{var2}^({random.choice([powprod,powsum])})",
                        f"({random.choice([cprod,csum])}{var1}{var2})^({random.choice([powsum,powprod])})"]]
    #Now there is a standard format 3a x 4d x 5ad
    else:
        return[[QStart + ' ' + Q, Q],
                       [f"{cprod}{var1}^({pow1+pow3[0]}){var2}^({pow2+pow3[1]})",
                        f"{csum}{var1}^({pow1+pow3[0]}){var2}^({pow2+pow3[1]})",
                        f"{cprod*rneg}({var1}{var2})^({random.choice([powprod,powsum])})",
                        f"{csum*rneg}({var1}{var2})^({random.choice([powprod*rneg,powsum*rneg])})",
                        f"({random.choice([cprod,csum])*rneg}{var1}{var2})^({random.choice([powsum,powprod])})"]]
        
    
#Style 3 style 2, but terms have a power of 1-8 with 2 being more likely.
#Style 4 style 2, but there are now three terms, one of which is an 5ab or 7cd or something like this. (focus on a, b, c only)
#Style 5 Now terms or powers may be negative, and answers as well.

    

def QGen_MultiplyTerms(NumOfQs, DiffList):
    qps = SetQPS(NumOfQs,DiffList)
    OutQ = [[],[],[]]
    SentStarter = ['Simplify ','Collect these like terms: ']
    Variables = ['a','b','c','d','e','f','x','y','z']
    Ans = ''
    DiffListCounter = min(DiffList)
    
    while DiffListCounter < 6:
        if DiffListCounter in DiffList:
            for z in range(qps[0]):
                Question = Style_MultiplyTerms(DiffListCounter)
                Ans = Question[1][0]
                MC = NoneStatementChance(Question[1])
                MCAns = MC[0] #Small chance this could now be the nonestatement
                MC = Shuffle(MC)
                MCAns = IndexToABC(MC.index(MCAns)) #Taking the answer, converting to index, then converting to a letter.
                #appends
                OutQ[0].append([Question[0][0], Question[0][1]])
                OutQ[1].append(MC)
                OutQ[2].append([Ans,MCAns])
            qps.pop(0)
        DiffListCounter += 1
    return OutQ
            
def QGen_AddMultTerms(NumOfQs, DiffList):

    LocalDiffList = DiffList.copy()
    a = QGen_CollectLikeTerms(math.floor(NumOfQs/2), LocalDiffList)
    LocalDiffList = DiffList.copy()
    b = QGen_MultiplyTerms(NumOfQs - math.floor(NumOfQs/2), LocalDiffList)

    return blend([a,b])

In [61]:
#QGen_AddNegatives
#Style 1: Arrange in order from smallest to largest
#Style 2: small - big i.e. 4 - 10, 6 - 11
#Style 3: QF.MakeAddQuestion with a positive and a negativeQF
#Style 4: QF.MakeAddQuestion completely randomised numbers, both neg both pos whatever
#Style 5: Contains 3 terms, with chance of one being a square i.e. 8^2
#NOTE THESE ARE ALL 4 OPTIONS NOT 5

def Style_AddNegatives(StyleInt):
    if StyleInt == 1:
        #Style 1: Arrange in order from smallest to largest. a smallest d largest.
        a = random.randint(-50,-20) - random.choice([random.randint(1,9)/10,0])
        b = a + random.choice([random.randint(1,9)/10,random.randint(1,9),random.randint(1,9)/10 + random.randint(1,9)])
        c = random.randint(0,math.floor(a*-1)-5)
        d = a*-1 - random.randint(1,3) + random.randint(1,9)/10
        abcd = [a, b, c, d]
        random.shuffle(abcd)
        abcdstr = f"{abcd[0]},  {abcd[1]},  {abcd[2]},  {abcd[3]}"
        if random.choice([True,False]):
            return [[f"What is the largest number in {abcdstr}?",f"Largest in {abcdstr}?"],[d,a,b,c]]
        else:
            return [[f"What is the smallest number in {abcdstr}?",f"Smallest in {abcdstr}?"],[a,b,c,d]]
    
    if StyleInt == 2 or StyleInt ==3:
        a = random.randint(2,30)*random.choice([1,1,1,1,2,3,5,10])
        b = a + random.randint(1,random.choice([5,10,50]))
        if StyleInt==2:
            return [[f"Calculate {a} - {b} =",f"{a} - {b} ="],[a-b,a+b,-a-b,b-a]]
        else:
            b = b*-1
            Question = QF.MakeAddQuestion(a,b)
            if "." in Question[1]:
                Ans = float(Question[1])
            else:
                Ans = int(Question[1])
            OutMCQs = list(set([Ans,a,b,a+b,a-b,b-a]))
            if Ans in OutMCQs:
                index = OutMCQs.index(Ans)
                OutMCQs.pop(index)
            OutMCQs = [Ans] + OutMCQs
            while len(OutMCQs) >= 5:
                OutMCQs.pop(random.randint(1,len(OutMCQs)-1))
            return [[f"Work out {Question[0]}",Question[0]],OutMCQs]
    
    if StyleInt == 4:
        a = random.randint(2,50)*random.choice([random.randint(1,10),random.randint(-10,-1),1,-1,10,-10])
        b = random.randint(2,50)*random.choice([random.randint(1,10),random.randint(-10,-1),1,-1,10,-10])
        if a>0 and b>0:
            a = a * random.choice([-1,1])
            b = b * random.choice([-1,1])
        Question = QF.MakeAddQuestion(a,b)
        if "." in Question[1]:
            Ans = float(Question[1])
        else:
            Ans = int(Question[1])
        OutMCQs = list(set([Ans,Ans*-1,a,b,a+b,(a+b)*-1,a-b,b-a]))
        if Ans in OutMCQs:
            index = OutMCQs.index(Ans)
            OutMCQs.pop(index)
        OutMCQs = [Ans] + OutMCQs
        while len(OutMCQs) >= 5:
            OutMCQs.pop(random.randint(1,len(OutMCQs)-1))
        return [[f"Calculate {Question[0]}",Question[0]],OutMCQs]
    
    else:
        a = random.randint(2,15)*random.choice([random.randint(1,10),random.randint(-10,-1),1,-1,10,-10])
        b = random.randint(2,15)*random.choice([random.randint(1,10),random.randint(-10,-1),1,-1,10,-10])/random.choice([1,10])
        c = random.randint(2,15)*random.choice([random.randint(1,10),random.randint(-10,-1),-1,-1,-10,-10])
        if random.choice([True,False]):
            return [[f"Work out {a} + {b} + {c}",f"{a} + {b} + {c} ="],[a+b+c,a-(b+c),a+b-c,b+c-a]]
        else:
            return [[f"Work out {a} - {b} + {c}",f"{a} + {b} + {c} ="],[a-b+c,a-(b+c),a+b-c,b+c-a]]

def QGen_AddNegatives(NumOfQs, DiffList):
    qps = SetQPS(NumOfQs,DiffList)
    OutQ = [[],[],[]]
    Ans = ''
    DiffListCounter = min(DiffList)
    
    while DiffListCounter < 6:
        if DiffListCounter in DiffList:
            for z in range(qps[0]):
                Question = Style_AddNegatives(DiffListCounter)
                Ans = Question[1][0]
                MC = Shuffle(Question[1])
                MCAns = IndexToABC(MC.index(Ans)) #Taking the answer, converting to index, then converting to a letter.
                #appends
                OutQ[0].append([Question[0][0], Question[0][1]])
                OutQ[1].append(MC)
                OutQ[2].append([Ans,MCAns])
            qps.pop(0)
        DiffListCounter += 1
    return OutQ

#QGen_TimesNegatives
#Style 1: pos by neg (a*b, a*b*-1, a + b, (a+b)*-1)
#Style 2: style 1 but random chance for a to be neg as well
#Style 3: style 2 but with QF.MakeMultQuestion(a,b)
#Style 4: multiply 3 things
#Style 5: a * b + c * d where any of a, b, c, d can be negative

def Style_TimesNegatives(StyleInt):
    sentstart = random.choice(['Work out ','Calculate ', 'Evaluate '])
    if StyleInt <= 2:
        #Style 1: pos by neg (a*b, a*b*-1, a + b, (a+b)*-1)
        a = random.randint(1,10)
        b = random.randint(-15,-1)
        c = random.choice([1,-1])
        if random.choice([True,False]):
            if StyleInt == 2:
                a = a*c
            return [[f"{sentstart}{a} × {b}",f"{a} × {b} ="],[a*b,a*b*-1,a+b,random.choice([a-b,b-a])]]
        else:
            if StyleInt==2:
                a = a*random.choice([1,-1])
                b = b*random.choice([1,-1])
                if a>0 and b>0:
                    a = a*-1
            else:
                a = a*c
                b = b*c
            return [[f"{sentstart}{a*b} ÷ {a}",f"{a*b} ÷ {a} ="],[b,b*-1,a*b - a,a*b + a]]
    
    a = random.randint(-15,15)
    b = random.randint(-20,20)
    if a==0:
        a += 5
    if b==0:
        b += 5
    if a>0 and b>0:
        a = a * random.choice([1,-1])
        b = b * random.choice([1,-1])
    
    if StyleInt == 3:    
        Question = QF.MakeMultQuestion(a,b)
        iterates = 0
        OutMCQs = []
        while len(set(OutMCQs)) < 4:
            c = random.choice([a*b-a,a*b-b,a*b+a,a*b+b,b+a])
            OutMCQs = [int(Question[1]),int(Question[1])*-1,c,c*-1]
            iterates += 1
            if iterates >= 2:
                print(f"Style_TimesNegative on StyleInt == 3. Iterates now at {iterates}")
        return [[f"{sentstart}{Question[0]}",Question[0]],OutMCQs]
    
    if StyleInt == 4:
        #b must now be a factor of a.
        b = random.randint(2,12)*random.choice([1,-1])
        a = b * random.randint(2,10)* random.choice([1,-1])
        c = random.randint(1,5)*random.choice([1,-1])
        return [[f"{sentstart}{a*c} ÷ {c} × {b}",f"{a*c} ÷ {c} × {b}"],[a*b,a*b*-1,int(a/b),int(a/b*-1)]]

    else:
        #Style 5: a * b + c * d where any of a, b, c, d can be negative
        a = random.randint(2,15)*random.choice([1,-1])
        b = random.randint(2,15)*random.choice([1,-1])
        c = random.randint(2,15)*random.choice([1,-1])
        d = random.randint(2,15)*random.choice([1,-1])
        return [[f"{sentstart}{a} × {b} + {c} × {d}",f"{a} × {b} + {c} × {d}"],[a*b + c*d,a*b*-1 - c*d,a*(b+c)*d,a*(b+c)*d*-1]]

def QGen_TimesNegatives(NumOfQs, DiffList):
    qps = SetQPS(NumOfQs,DiffList)
    OutQ = [[],[],[]]
    Ans = ''
    DiffListCounter = min(DiffList)
    
    while DiffListCounter < 6:
        if DiffListCounter in DiffList:
            for z in range(qps[0]):
                Question = Style_TimesNegatives(DiffListCounter)
                Ans = Question[1][0]
                MC = Shuffle(Question[1])
                MCAns = IndexToABC(MC.index(Ans)) #Taking the answer, converting to index, then converting to a letter.
                #appends
                OutQ[0].append([Question[0][0], Question[0][1]])
                OutQ[1].append(MC)
                OutQ[2].append([Ans,MCAns])
            qps.pop(0)
        DiffListCounter += 1
    return OutQ

def QGen_AddTimesNegatives(NumOfQs, DiffList):

    LocalDiffList = DiffList.copy()
    a = QGen_AddNegatives(math.floor(NumOfQs/2), LocalDiffList)
    LocalDiffList = DiffList.copy()
    b = QGen_TimesNegatives(NumOfQs - math.floor(NumOfQs/2), LocalDiffList)

    return blend([a,b])

In [66]:
#MakeQuiz('Adding and Multiplying Negatives',20,1,True,[1,2,3,4,5],'demo10')


Work out (-120) + 6.0 + 60
Work out (-120) + 6 + 60 
186.0
186 
-54.0
-54 
-186.0
-186 
-174.0
-174 
Work out 30 – 54.0 + 35
Work out 30 – 54 + 35 
11.0
11 
49.0
49 
-59.0
-59 
59.0
59 
-34.0
-34 
Work out (-14) – 13.0 + (-60)
Work out (-14) – 13 + (-60) 
59.0
59 
33.0
33 
-87.0
-87 
-33.0
-33 
-39.0
-39 
39.0
39 
-141.0
-141 
21.0
21 
Document demo10.docx has been created.
